In [1]:
# David Ouyang 12/5/2019

# Notebook which:
# 1. Downloads weights
# 2. Initializes model and imports weights
# 3. Performs test time evaluation of videos (already preprocessed with ConvertDICOMToAVI.ipynb)

import re
import os, os.path
from os.path import splitext
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import time
import subprocess
import datetime
from datetime import date
import sys
import cv2
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math
import torch
import torchvision
import echonet
import wget 

destinationFolder = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\Output"
videosFolder = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\a4c-video-dir"
DestinationForWeights = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\EchoNetDynamic-Weights"

In [2]:
# Download model weights

if os.path.exists(DestinationForWeights):
    print("The weights are at", DestinationForWeights)
else:
    print("Creating folder at ", DestinationForWeights, " to store weights")
    os.mkdir(DestinationForWeights)
    
segmentationWeightsURL = 'https://github.com/douyang/EchoNetDynamic/releases/download/v1.0.0/deeplabv3_resnet50_random.pt'
ejectionFractionWeightsURL = 'https://github.com/douyang/EchoNetDynamic/releases/download/v1.0.0/r2plus1d_18_32_2_pretrained.pt'


if not os.path.exists(os.path.join(DestinationForWeights, os.path.basename(segmentationWeightsURL))):
    print("Downloading Segmentation Weights, ", segmentationWeightsURL," to ",os.path.join(DestinationForWeights,os.path.basename(segmentationWeightsURL)))
    filename = wget.download(segmentationWeightsURL, out = DestinationForWeights)
else:
    print("Segmentation Weights already present")
    
if not os.path.exists(os.path.join(DestinationForWeights, os.path.basename(ejectionFractionWeightsURL))):
    print("Downloading EF Weights, ", ejectionFractionWeightsURL," to ",os.path.join(DestinationForWeights,os.path.basename(ejectionFractionWeightsURL)))
    filename = wget.download(ejectionFractionWeightsURL, out = DestinationForWeights)
else:
    print("EF Weights already present")
        


The weights are at C:\Users\Windows\Dropbox\Echo Research\CodeBase\EchoNetDynamic-Weights
Segmentation Weights already present
EF Weights already present


In [3]:
# Initialize and Run EF model

frames = 32
period = 2
batch_size = 20
model = torchvision.models.video.r2plus1d_18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)


device = torch.device("cuda")
if device.type == "cuda":
    model = torch.nn.DataParallel(model)
model.to(device)

# try some random weights: final_r2+1d_model_regression_EF_sgd_skip1_32frames.pth.tar
# scp ouyangd@arthur2:~/Echo-Tracing-Analysis/final_r2+1d_model_regression_EF_sgd_skip1_32frames.pth.tar "C:\Users\Windows\Dropbox\Echo Research\CodeBase\EchoNetDynamic-Weights"
#Weights = "final_r2+1d_model_regression_EF_sgd_skip1_32frames.pth.tar"

print("loading weights from ", os.path.join(DestinationForWeights, "r2plus1d_18_32_2_pretrained"))

checkpoint = torch.load(os.path.join(DestinationForWeights, os.path.basename(ejectionFractionWeightsURL)))
model.load_state_dict(checkpoint['state_dict'])

output = os.path.join(destinationFolder, "ef_output.csv")

ds = echonet.datasets.Echo(split = "external_test", external_test_location = videosFolder, crops="all")
print(ds.split, ds.fnames)

mean, std = echonet.utils.get_mean_and_std(ds)

kwargs = {"target_type": "EF",
          "mean": mean,
          "std": std,
          "length": frames,
          "period": period,
          }

ds = echonet.datasets.Echo(split = "external_test", external_test_location = videosFolder, **kwargs, crops="all")

test_dataloader = torch.utils.data.DataLoader(ds, batch_size = 1, num_workers = 5, shuffle = True, pin_memory=(device.type == "cuda"))
loss, yhat, y = echonet.utils.video.run_epoch(model, test_dataloader, "test", None, device, save_all=True, blocks=25)

with open(output, "w") as g:
    for (filename, pred) in zip(ds.fnames, yhat):
        for (i,p) in enumerate(pred):
            g.write("{},{},{:.4f}\n".format(filename, i, p))


loading weights from  C:\Users\Windows\Dropbox\Echo Research\CodeBase\EchoNetDynamic-Weights\r2plus1d_18_32_2_pretrained
external_test ['0X1A05DFFFCAFB253B.avi', '0X1A0A263B22CCD966.avi', '0X1A2A76BDB5B98BED.avi', '0X1A2C60147AF9FDAE.avi', '0X1A2E9496910EFF5B.avi', '0X1A3D565B371DC573.avi', '0X1A3E7BF1DFB132FB.avi', '0X1A5FAE3F9D37794E.avi', '0X1A6ACFE7B286DAFC.avi', '0X1A8D85542DBE8204.avi']


100%|██████████| 10/10 [00:32<00:00,  3.20s/it, 3524.12 (4518.84) / 0.00]
